# NS‑3 Code Generator with **Gemini 2.0 Flash**
This notebook batch‑generates ns‑3 C++ simulations from natural‑language descriptions.
It reads an *instruction prompt* from `prompt.yaml`, iterates over every `.txt` file found in `Dataset/Prompts/Large` and `Dataset/Prompts/Small`, and writes the generated code to `BasicEvaluation/Gemini/<name>.cc`.

Run the cells top‑to‑bottom.

## ⚙️ Install Python dependencies

In [ ]:
%pip install -q -r ../requirements.txt

## 📦 Imports && logging setup

In [ ]:
from pathlib import Path
import os, re, yaml, logging
from dotenv import load_dotenv
import google.generativeai as genai
from rich.logging import RichHandler
from tqdm import tqdm
import time

logging.basicConfig(level='INFO', handlers=[RichHandler()], format='%(message)s')

## 🛠 Configuration

In [ ]:
PROMPT_YAML = Path('../prompt.yaml')
SRC_DIRS = [Path('../Dataset/Prompts/Large'), Path('../Dataset/Prompts/Small')]
OUT_DIR  = Path('Basic/Gemini')
MODEL_NAME = 'gemini-2.5-flash-preview-05-20'
CODE_BLOCK_RE = re.compile(r"```(?:cpp|c\+\+)?\s*([\s\S]+?)```", re.IGNORECASE)

### Helper — extract first fenced C++ block

In [ ]:
def extract_code(text: str) -> str:
    m = CODE_BLOCK_RE.search(text)
    return m.group(1).strip() if m else text.strip()

## 📑 Load instruction prompt

In [ ]:
instruction_prompt = yaml.safe_load(PROMPT_YAML.open())['instruction_prompt'].strip()
print(instruction_prompt)

## 🔑 Authenticate & initialise Gemini

In [ ]:
load_dotenv()
assert 'GEMINI_API_KEY' in os.environ, 'Add GEMINI_API_KEY to your .env file'
genai.configure(api_key=os.environ['GEMINI_API_KEY'])
model = genai.GenerativeModel(MODEL_NAME)
print('Model ready!')

## 🗂 Collect description files

In [ ]:
txt_files = []
for d in SRC_DIRS:
    txt_files.extend(sorted(d.glob('*.txt')))
print(f'Found {len(txt_files)} description files')
OUT_DIR.mkdir(parents=True, exist_ok=True)

## 🚀 Generate ns‑3 code

In [ ]:
for src in tqdm(txt_files, desc='Generating'):
    description = src.read_text(encoding='utf-8').strip()
    prompt = f"{instruction_prompt}\n\nProgram description:\n\"\"\"\n{description}\n\"\"\""
    try:
        resp = model.generate_content(prompt)
        code = extract_code(resp.text)
        outfile = OUT_DIR / f"{src.stem}.cc"
        outfile.write_text(code, encoding='utf-8')
        logging.info(f'✓ {src.name} → {outfile}')
    except Exception as e:
        logging.error(f'✗ {src.name} – {e}')

    time.sleep(7)